![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Proyecto 2 - Clasificación de género de películas

El propósito de este proyecto es que puedan poner en práctica, en sus respectivos grupos de trabajo, sus conocimientos sobre técnicas de preprocesamiento, modelos predictivos de NLP, y la disponibilización de modelos. Para su desarrollo tengan en cuenta las instrucciones dadas en la "Guía del proyecto 2: Clasificación de género de películas"

**Entrega**: La entrega del proyecto deberán realizarla durante la semana 8. Sin embargo, es importante que avancen en la semana 7 en el modelado del problema y en parte del informe, tal y como se les indicó en la guía.

Para hacer la entrega, deberán adjuntar el informe autocontenido en PDF a la actividad de entrega del proyecto que encontrarán en la semana 8, y subir el archivo de predicciones a la [competencia de Kaggle](https://www.kaggle.com/t/29c44fce98c747f2a1dfdaf29d4c4965).

## Datos para la predicción de género en películas

![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/moviegenre.png)

En este proyecto se usará un conjunto de datos de géneros de películas. Cada observación contiene el título de una película, su año de lanzamiento, la sinopsis o plot de la película (resumen de la trama) y los géneros a los que pertenece (una película puede pertenercer a más de un género). Por ejemplo:
- Título: 'How to Be a Serial Killer'
- Plot: 'A serial killer decides to teach the secrets of his satisfying career to a video store clerk.'
- Generos: 'Comedy', 'Crime', 'Horror'

La idea es que usen estos datos para predecir la probabilidad de que una película pertenezca, dada la sinopsis, a cada uno de los géneros.

Agradecemos al profesor Fabio González, Ph.D. y a su alumno John Arevalo por proporcionar este conjunto de datos. Ver https://arxiv.org/abs/1702.01992

## Ejemplo predicción conjunto de test para envío a Kaggle
En esta sección encontrarán el formato en el que deben guardar los resultados de la predicción para que puedan subirlos a la competencia en Kaggle.

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# Importación librerías
import pandas as pd
import os
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from xgboost import XGBRegressor, XGBClassifier
from sklearn.metrics import r2_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [6]:
# Carga de datos de archivo .csv
dataTraining = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTraining.zip', encoding='UTF-8', index_col=0)
dataTesting = pd.read_csv('https://github.com/albahnsen/MIAD_ML_and_NLP/raw/main/datasets/dataTesting.zip', encoding='UTF-8', index_col=0)

In [7]:
# Visualización datos de entrenamiento
dataTraining.head()

,year,title,plot,genres,rating
3107,2003,Most,most is the story of a single father who takes...,"['Short', 'Drama']",8.0
900,2008,How to Be a Serial Killer,a serial killer decides to teach the secrets o...,"['Comedy', 'Crime', 'Horror']",5.6
6724,1941,A Woman's Face,"in sweden , a female blackmailer with a disfi...","['Drama', 'Film-Noir', 'Thriller']",7.2
4704,1954,Executive Suite,"in a friday afternoon in new york , the presi...",['Drama'],7.4
2582,1990,Narrow Margin,"in los angeles , the editor of a publishing h...","['Action', 'Crime', 'Thriller']",6.6


In [5]:
# Visualización datos de test
dataTesting.head()

,year,title,plot
1,1999,Message in a Bottle,"who meets by fate , shall be sealed by fate ...."
4,1978,Midnight Express,"the true story of billy hayes , an american c..."
5,1996,Primal Fear,martin vail left the chicago da ' s office to ...
6,1950,Crisis,husband and wife americans dr . eugene and mr...
7,1959,The Tingler,the coroner and scientist dr . warren chapin ...


#### Ejemplo profesor

In [6]:
# Definición de variables predictoras (X)
vect = CountVectorizer()
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 38672)

In [17]:
# Definición de variable de interés (y)
dataTraining['genres'] = dataTraining['genres'].map(lambda x: eval(x))
le = MultiLabelBinarizer()
y_genres = le.fit_transform(dataTraining['genres'])

In [8]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [9]:
# Definición y entrenamiento
clf = OneVsRestClassifier(RandomForestClassifier(n_jobs=-1, n_estimators=100, max_depth=10, random_state=42))
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=RandomForestClassifier(max_depth=10, n_jobs=-1,
                                                     random_state=42))

In [10]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.7967558619238986

#### Utilizando Lemmatization y stopwords

In [8]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')

# Descargar recursos de nltk si aún no están
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

**Opción vista en clase**

In [10]:
# Lista de stopwords en inglés
stop_words = set(stopwords.words('english'))

#Lematizador
wordnet_lemmatizer = WordNetLemmatizer()

# Función de lematización + eliminación de stopwords
def split_lemmas_no_stopwords(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    lemmas = [wordnet_lemmatizer.lemmatize(token, pos='a') for token in tokens if token not in stop_words]
    return lemmas

**Opción propuesta por Gemini IA**

In [11]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Download necessary NLTK data (run these lines once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') # Needed for nltk.pos_tag
nltk.download('omw-1.4')

wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

#Convert NLTK treebank tags to WordNet tags for lemmatization.
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN # Default to noun if POS tag is not found or is ambiguous

def split_lemmas_no_stopwords(text):
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    # Filter out punctuation and numbers, then POS tag
    filtered_tokens = [word for word in tokens if word.isalpha()]
    pos_tagged_tokens = nltk.pos_tag(filtered_tokens)

    lemmas = []
    for word, tag in pos_tagged_tokens:
        if word not in stop_words:
            # Get the WordNet POS tag
            wn_tag = get_wordnet_pos(tag)
            lemma = wordnet_lemmatizer.lemmatize(word, pos=wn_tag)
            lemmas.append(lemma)

    return lemmas

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\junio\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


**CountVectorizer**

In [14]:
vect = CountVectorizer(analyzer=split_lemmas_no_stopwords, ngram_range=(1,2))
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 32360)

**TfidfVectorizer**

In [15]:
vect = TfidfVectorizer(analyzer=split_lemmas_no_stopwords, ngram_range=(1,2))
X_dtm = vect.fit_transform(dataTraining['plot'])
X_dtm.shape

(7895, 32360)

In [18]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test usandola función train_test_split
X_train, X_test, y_train_genres, y_test_genres = train_test_split(X_dtm, y_genres, test_size=0.33, random_state=42)

In [19]:
X_train.shape

(5289, 32360)

#### **Modelos usados**

In [20]:
clf = OneVsRestClassifier(LogisticRegression())
clf.fit(X_train, y_train_genres)

OneVsRestClassifier(estimator=LogisticRegression())

In [21]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.878821708944821

Archivo 6:  
clf = OneVsRestClassifier(LogisticRegression())
Utilizando vect = TfidfVectorizer(analyzer=split_lemmas_no_stopwords, ngram_range=(1,2)) y el lematizador de Geminii

In [25]:
# Predicción del modelo de clasificación
y_pred_genres = clf.predict_proba(X_test)

# Impresión del desempeño del modelo
roc_auc_score(y_test_genres, y_pred_genres, average='macro')

0.878821708944821

Archivo 5:  
clf = OneVsRestClassifier(LogisticRegression())  
Utilizando vect = TfidfVectorizer(analyzer=split_lemmas_no_stopwords, ngram_range=(1,2))

In [100]:
# transformación variables predictoras X del conjunto de test
X_test_dtm = vect.transform(dataTesting['plot'])

cols = ['p_Action', 'p_Adventure', 'p_Animation', 'p_Biography', 'p_Comedy', 'p_Crime', 'p_Documentary', 'p_Drama', 'p_Family',
        'p_Fantasy', 'p_Film-Noir', 'p_History', 'p_Horror', 'p_Music', 'p_Musical', 'p_Mystery', 'p_News', 'p_Romance',
        'p_Sci-Fi', 'p_Short', 'p_Sport', 'p_Thriller', 'p_War', 'p_Western']

# Predicción del conjunto de test
y_pred_test_genres = clf.predict_proba(X_test_dtm)

In [101]:
y_pred_test_genres[0]

array([0.09726071, 0.10030135, 0.02451814, 0.02702917, 0.41055725,
       0.11398006, 0.0287612 , 0.5874855 , 0.05351459, 0.11723927,
       0.02152927, 0.02142835, 0.0920194 , 0.02963565, 0.03757091,
       0.1041462 , 0.00066875, 0.58686968, 0.05968723, 0.01198021,
       0.02428666, 0.194627  , 0.02616347, 0.0292456 ])

In [102]:
# Guardar predicciones en formato exigido en la competencia de kaggle
ruta ='D:/User/Documentos/Maestría MIAD/Ciclo 4/Machine Larning y PLN/'

res = pd.DataFrame(y_pred_test_genres, index=dataTesting.index, columns=cols)
res.to_csv(ruta+'pred_genres_g29_6.csv', index_label='ID')
res.head()

,p_Action,p_Adventure,p_Animation,p_Biography,p_Comedy,p_Crime,p_Documentary,p_Drama,p_Family,p_Fantasy,...,p_Musical,p_Mystery,p_News,p_Romance,p_Sci-Fi,p_Short,p_Sport,p_Thriller,p_War,p_Western
1,0.097261,0.100301,0.024518,0.027029,0.410557,0.113980,0.028761,0.587485,0.053515,0.117239,...,0.037571,0.104146,0.000669,0.586870,0.059687,0.011980,0.024287,0.194627,0.026163,0.029246
4,0.179420,0.046375,0.032940,0.101630,0.272385,0.372117,0.056087,0.728662,0.038777,0.028497,...,0.032332,0.040600,0.000753,0.085010,0.030889,0.014265,0.026535,0.260054,0.047831,0.034408
5,0.090063,0.041879,0.017553,0.045754,0.190257,0.550218,0.029942,0.771803,0.030784,0.039498,...,0.022920,0.313905,0.000670,0.185668,0.069343,0.009340,0.022440,0.479815,0.034465,0.022689
6,0.125949,0.095002,0.019694,0.036235,0.245323,0.096399,0.030970,0.691485,0.059585,0.053041,...,0.031474,0.084354,0.000679,0.287917,0.088948,0.009071,0.029795,0.324779,0.066513,0.022856
7,0.077295,0.073068,0.027868,0.027672,0.223309,0.100046,0.038127,0.285421,0.058955,0.129188,...,0.025918,0.149885,0.000684,0.159820,0.360185,0.010890,0.017211,0.324091,0.024422,0.021779
